In [1]:
from annif.corpus.document import  DocumentDirectory
from annif.corpus.skos import SubjectFileSKOS
from annif.corpus.subject import SubjectIndex
import rrflow
import rrflow.utils
import nltk
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec
import typing
from collections import defaultdict
import math


In [2]:
articles_path = '/home/nots/temp/annif/'
vocab_file_path = '/home/nots/temp/dbc-hierarchy.ttl'

In [4]:
#list(dd.documents)[:10]

In [5]:
subjects = SubjectFileSKOS(vocab_file_path)

In [5]:
#subjects

In [6]:
subject_index = SubjectIndex()

In [7]:
subject_index.load_subjects(subjects)

In [9]:
dd = DocumentDirectory(path=articles_path, subject_index=subject_index, language='da', require_subjects=True)

In [9]:
#subject_index[253]
#subject_index.get(6765)

In [10]:
d = next(x for i,x in enumerate(dd.documents) if i==7)
d.subject_set._subject_ids


[10113, 12890, 13495]

In [33]:
#import textwrap
#print(textwrap.fill(d.text, 120))

In [11]:
subject_index._subjects[13495].labels['da']

'entrepriser, forretninger, erhvervsdrivende virksomheder, virksomheder, selskaber'

In [12]:
corpus = {}
subject_dict = {}
doc_freq = defaultdict(int)
idf_dict = defaultdict(float)
for doc_index, d in enumerate(dd.documents):
    content = d.text
    clean_content = rrflow.utils.clean_article_content(content)
    words = nltk.tokenize.word_tokenize(clean_content, language='danish')
    corpus[doc_index] = TaggedDocument(words=words, tags=[doc_index])
    subject_dict[doc_index] = d.subject_set._subject_ids
    for subject_id in d.subject_set._subject_ids:
        doc_freq[subject_id] += 1
    article_count = doc_index
idf_dict = {s:math.log(article_count/(1 + doc_freq[s])) for s in doc_freq.keys()}
model = Doc2Vec(corpus.values())


In [19]:
def get_article_nearest_neighbors(article_text: str, limit:int=10, model: Doc2Vec=None) -> typing.List[typing.Tuple[int,float]]:
    """
    Returns the embedding of the given article.
    """
    art_clean_content = rrflow.utils.clean_article_content(article_text)
    words = nltk.tokenize.word_tokenize(art_clean_content, language='danish')
    embedding = model.infer_vector(words)
    return model.dv.most_similar([embedding], topn=limit)

def get_article_subjects(article_text: str, limit:int=5, model:Doc2Vec=None) -> typing.Dict[str, float]:
    """
    Returns a dict of subjects and their weights.
    """
    nearest_neighbors = get_article_nearest_neighbors(article_text, limit, model)
    subjects_dict = defaultdict(float)
    for article_id, distance in nearest_neighbors:
        article_subject_ids = subject_dict[article_id]
        for a_subj_index in article_subject_ids:
            subject_text = subject_index._subjects[a_subj_index].labels['da']
            tf = (min(article_text.count(subject_text),1) / len(article_text)) if len(article_text) > 0 else 0
            idf = idf_dict[a_subj_index]
            subjects_dict[a_subj_index] += (1 / distance) * (tf / idf)
    res = dict(subjects_dict)
    return [(s, res[s]) for s in sorted(res, key=res.get, reverse=True)[:limit]]




In [23]:
get_article_nearest_neighbors(val_article_text, limit=10, model=model)

[(21575, 0.7433757781982422),
 (2492, 0.7213847041130066),
 (81613, 0.676607608795166),
 (74339, 0.6693227291107178),
 (68518, 0.6668237447738647),
 (69373, 0.6612148880958557),
 (29055, 0.6580134630203247),
 (12507, 0.6570197939872742),
 (86844, 0.6559286117553711),
 (61045, 0.6559083461761475)]

In [30]:
[(subject_index._subjects[p[0]].labels['da'],p[1]) for p in get_article_subjects(val_article_text, limit=5, model=model)]

[('genetik', 8.676668036729721e-05),
 ('proteiner', 2.1708377532912718e-05),
 ('biologi, biologisme', 0.0),
 ('genforskning', 0.0),
 ('research, efterforskning, forskning', 0.0)]

In [29]:
val_article_text = """
Jeg ville aldrig have gættet, hvad der var inde i fødselsdagsgaven fra min mor. Pakken var firkantet, men for let til at være en bog og for flad til at være et af de der gavesæt med parfume, sæbe og creme. Så jeg var forberedt på lidt af hvert. Men da jeg pillede papiret af, blev jeg alligevel overrasket.I en kasse lå et komplet sæt til at lave en DNA-test og sende den til et laboratorium i Texas. DNA er den arvemasse, vi har fået fra vores forældre. Det findes indeni vores celler, og DNA bestemmer, hvilke synlige og usynlige træk vi arver fra vores forfædre. Når specialister analyserer en persons DNA, kan de finde ud af, hvilke forskellige folkeslag forfædrene kom fra.»Jeg har også købt en til mig selv, er det ikke spændende?« udbrød min mor ivrigt.»Øh jooo,« sagde jeg og kiggede på hendes fregnede hud og blå øjne, som ligner mine egne. Ligner min morfars. Jeg tænkte for mig selv, at testen nok bare ville fortælle, at vi er 100 procent kartoffeldanskere og vikinger.Først skulle vi tage en DNA-prøve fra os selv. Pakken gemte på et lille klinisk kit med to små rør med skruelåg. De skulle skrues af og på med helt rene hænder. Det ville jo være uheldigt, hvis et kattehår fik testen til at vise nogle meget mærkelige resultater. I en plasticpakke lå noget, der lignede vatpinde, bare længere og med en takket dut for enden.De skulle bruges til at skrabe på indersiden af kinden. Min mor skrabede så ivrigt, at hendes pind blev lyserød af blod.Så skulle dutterne ned i rørene og knækkes af pinden. På med skruelåget og ned i en foret konvolut, som fragtede vores kind-snask sikkert til USA. På laboratoriet i Texas trak de vores DNA frem fra de celler, vi havde sendt af sted. Derefter sammenlignede de vores DNA med registreret DNA fra mange forskellige lande for at finde de steder, hvor der er det største match.Røskva, du er..I Texas er det højlys dag, mens det stadig er midt om natten i Danmark. Derfor ligger min telefon og blinker med en e-mail, da jeg vågner en tidlig morgen. Stadig halvt i søvne får jeg åbnet mailen, og ord toner langsomt frem på skærmen.»Røskva, du er...« står der, mens der spiller spændingsfuld musik.»70 procent skandinav...« Den del er ikke så overraskende.»21,2 procent englænder...« Det giver mening. Vikingerne rejste jo til og fra England.»Otte procent græker...« Nå, det er da lidt sjovt. Nu kan der da ikke være mere tilbage? »Og 0,8 procent indianer.« Jeg sætter mig op i sengen med et sæt.Hvad? Indianer? Fra Indien? Eller indianer som de første amerikanere? Et kort viser sig på telefonens skærm, og der er ganske rigtigt en ring omkring Nordamerika.Jeg stavrer ud på badeværelset og kigger mig i spejlet. Med mit rødbrune morgenhår og min lyse hud med lagenmærker på kinden er det svært lige at få øje på de græske og indianske gener.1,5 procent iberiskMin mor er knap så begejstret for sit resultat.Det nordamerikanske DNA stammer nemlig ikke fra hende, viser det sig. Alle børn arver halvdelen af deres DNA fra moren og den anden halvdel fra deres far. Forældrene har også arvet fra deres mor og far, men i princippet kan man kun give DNA videre fra sin ene forælder. Det viser sig da også, at min mor har noget DNA, som ikke er gået i arv til mig.»Jeg er 1,5 procent iberisk,« siger hun i telefonen.Jeg ved ikke lige, hvad iberisk betyder.»Fra Iberia, altså Gibraltar. Ligesom de iberiske snegle - dræbersneglene - som er min værste fjende i haven,« siger hun og griner.På en hjemmeside sammenligner vi vores resultater. Siden bekræfter, at hun er min mor.Ved at få undersøgt min fars DNA kunne jeg finde endnu flere brikker i puslespillet. Men han gider ikke få lavet en DNA-test.»Det er sgu spild af penge. Det kan da være lige meget, hvor jeg kommer fra,« siger han bare.Det kan han selvfølgelig have ret i. Men for nogle betyder det meget at undersøge deres rødder. Nogle bruger det til at lede efter fjerne familiemedlemmer. En reklame på nettet bruger DNA-tests som eksempel på, at racisme er fjollet, fordi vi alle deler fælles forfædre, hvis man går langt nok tilbage.For mig er det en lidt blandet følelse.Resultatet siger jo ikke rigtig noget om, hvem jeg er. Men det er alligevel interessant at tænke på, hvordan mine forfædre mon mødte hinanden. Hvordan handelsrejser, hungersnød, krig og kærlighed kan have fået dem til at rejse over kontinenter og verdenshave. Hele den lange række af mennesker og tilfælde, som er grunden til, jeg findes. Men mest af alt rejser det en masse spørgsmål om, hvad DNA egentlig kan bruges til.Det store billedeRummet er et stort, lyst hjørnekontor på Naturhistorisk Museum i København. Det tilhører Eske Willerslev, som er professor ved Københavns Universitet på Center for Geogenetik. Han bruger DNA-spor til at opklare mysterier om menneskets fortid på Jorden.Et ældgammelt, massivt træbord bærer på papirer og en Apple-computer. På reolerne står bøger ved siden af et gulligt, halvsmuldret kranie fra et kolossalt, uddødt fortidsdyr.Eske Willerslev slår sig ned i lædersofaen, stikker et stykke snustobak under læben og tager en slurk fra sin kaffekop.»Indianer? Det er lidt interessant,« siger han og ser tænksom ud et øjeblik.Men hans interesse i mit eksotiske DNA er ikke så stor.»Problemet er, at de fleste forestiller sig, at den slags tests viser noget andet, end de nødvendigvis gør,« forklarer han.»Det kan godt være, at dit DNA virkelig er beslægtet med de mennesker, der bor i Grækenland i dag, eller med nordamerikanske indianere. Men det kan også være, at jeres slægtskab går meget længere tilbage til en tid, hvor jeres fælles forfædre boede et helt andet sted.« Han synes derfor, at den her slags test er lidt fjollede. Eske Willerslev er mere interesseret i det store billede. Hans forskning handler om at forstå vores forhistorie, og hvor vi kommer fra.»Vi udvinder den samme slags information, som de har gjort fra dig. Vi gør det bare fra fortidslevn. Og på den måde kan vi finde ud af, hvornår de forskellige fortidsmennesker har mødt hinanden, og hvordan de har bevæget sig rundt omkring på Jorden.« Det gør Eske Willerslev og hans kolleger ved at undersøge ældgamle rester af hår og tænder, som er fundet i arkæologiske udgravninger.Når forskerne sammenligner DNA fra mange forskellige skeletter fra et bestemt sted, kan de lave en tidslinje og regne ud, hvor og hvornår de forskellige folk har mødt hinanden.»Det kan din test ikke sige noget om. Men det gode ved den er til gengæld, at den viser, at ideen om forskellige racer er meningsløs. For vi har alle sammen miksede forfædre, som har mødt hinanden op igennem historien og fået børn sammen.«.Fakta: DNAAlt fra din øjenfarve til formen på din lilletå er bestemt af DNA. Navnet er en forkortelse af det engelske ord DeoxyriboNucleic Acid. Det beskriver det molekyle, som findes i kernen af de fleste celler i kroppen. Molekylet består af en masse atomer, formet som en slags snoet stige. Hvert trin på stigen indeholder informationer, der fortæller kemikalier i kroppen, hvordan de skal sætte sig sammen. Sådan bliver proteiner skabt, som dernæst samles til celler, der endelig bliver til det væv, som mennesker, dyr og planter er lavet af.Du får den ene halvdel af dit DNA fra din mor og den anden fra din far. Dit DNA er helt unikt for dig, for selv enæggede tvillinger har små forskelle. Analyser af en persons DNA kan bruges til at identificere dem, og det kan fortælle en masse, for eksempel om forfædre og hvilke sygdomme man måske kan få.OPDAGELSEN AF DNA1869 En læge fra Schweiz, Friedrich Miescher, opdager en sur væske indeni celler.Han kaldte det nuklein, men det var DNA.1928 Bakteriologen Frederick Griffith laver eksperimenter, der tyder på, at DNA er arvemassen, som indeholder vores gener. Det blev endeligt bevist i 1944 af to andre forskere.1952-53 Biofysikeren Rosalind Franklin tager de første røntgenkrystal-billeder af DNA. Billederne får molekylærbiologerne Francis Crick og James D. Watson til at opdage, at DNA er formet som en dobbelt-spiral.2003 Den komplette kode for et menneskes arvemasse bliver kortlagt. 2017 I dag arbejder mange DNA-eksperter på at finde ud af, hvordan DNA-forskning kan bruges til at forebygge og behandle sygdomme. 1866 Den østrigske munk Gregor Mendel skriver, at ærter arver særlige træk fra deres ærte-forfædre. Det er den første opdagelse af gener.1953-1996 Masser af forskere finder frem til de oplysninger, der ligger gemt i DNA. Sygdommes koder bliver opdaget, og det bliver muligt at lave kloner af dyr ud fra DNA. 2014 Forskere i USA skaber en organisme med kunstig DNA.
"""